In [1]:
import pandas as pd
import numpy as np

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM,Trainer, TrainingArguments, DataCollatorForLanguageModeling

In [3]:
from datasets import Dataset

In [15]:
tokenizer = AutoTokenizer.from_pretrained("DeepESP/gpt2-spanish-medium")
model = AutoModelForCausalLM.from_pretrained("LuisDavidFT777/gpt2-spanish-medium")

In [33]:
df = pd.read_csv('preguntas_respuesta')


In [35]:
 df.groupby('nombretitulo')['nombrearea'].agg(list)

nombretitulo
Abogado                                          [Humanidades y Sociales, Humanidades y Sociale...
Administrador Educativo                          [Humanidades y Sociales, Humanidades y Sociale...
Administrador Financiero                         [Ciencias Económicas, Ciencias Económicas, Cie...
Administrador de Empresas                        [Ciencias Económicas, Ciencias Económicas, Cie...
Analista de Sistemas                             [Tecnológicas, Tecnológicas, Tecnológicas, Tec...
                                                                       ...                        
Técnico Superior en Topografia                   [Tecnológicas, Tecnológicas, Tecnológicas, Tec...
Técnico Superior en Turismo                      [Humanidades y Sociales, Humanidades y Sociale...
Técnico Universitario Superior en Gastronomía    [Humanidades y Sociales, Humanidades y Sociale...
Técnico superior en Comunicación Radiofónica     [Humanidades y Sociales, Humanidades y Sociale.

In [38]:
df[df['nombrearea']=='Ciencias Agrarias']

,Unnamed: 0,nombreuniversidad,nombres,paterno,materno,nombretitulo,nombrecarrera,celular,nombrearea,fechanacimiento,fechatitulacion,edad,egreso,edad_rango,rango_ano_titulacion,nombrereemplazados,nombre_completo,pregunta,respuesta
4,4,AUTONOMA TOMAS FRIAS,MODESTO JORGE,DIAZ,ZELADA,Ingeniero en Recursos Naturales,Diplomado en Educación Superior V.7,71831895,Ciencias Agrarias,1964-06-15,1996-09-06,60,1996,47-mayor,antiguo,Diplomado en Educación Superior V.7,MODESTO JORGE DIAZ ZELADA,<humano> Dame una recomendacion de este curso ...,<titulo> Ingeniero en Recursos Naturales <area...
39,39,"MAYOR, REAL Y PONTIFICIA DE SAN FRANCISCO XAVIER",NESTOR,MIRANDA,INOCENTES,Ingeniero en Zootecnia,Diplomado en Educación Superior V.7,75437071,Ciencias Agrarias,1971-02-22,2003-11-28,53,2003,47-mayor,antiguo,Diplomado en Educación Superior V.7,NESTOR MIRANDA INOCENTES,<humano> Dame una recomendacion de este curso ...,<titulo> Ingeniero en Zootecnia <area> Ciencia...
61,61,"MAYOR, REAL Y PONTIFICIA DE SAN FRANCISCO XAVIER",REINALDO,LOZANO,AJATA,Ingeniero Agrónomo,Diplomado en Educación Superior V.7,NaN,Ciencias Agrarias,1980-10-14,2007-09-19,44,2007,40-46,antiguo,Diplomado en Educación Superior V.7,REINALDO LOZANO AJATA,<humano> Dame una recomendacion de este curso ...,<titulo> Ingeniero Agrónomo <area> Ciencias Ag...
67,67,AUTONOMA JUAN MISAEL SARACHO,ADOLFO RAMON,MUñOZ,OLLER,Ingeniero Agrónomo,Diplomado en Educación Superior V.7,71877812,Ciencias Agrarias,1961-11-07,1991-11-27,63,1991,47-mayor,antiguo,Diplomado en Educación Superior V.7,ADOLFO RAMON MUñOZ OLLER,<humano> Dame una recomendacion de este curso ...,<titulo> Ingeniero Agrónomo <area> Ciencias Ag...
70,70,"MAYOR, REAL Y PONTIFICIA DE SAN FRANCISCO XAVIER",LORGIO,SOLIZ,BALDERAS,Licenciado en Educación Ambiental,Diplomado en Educación Superior V.7,71157404,Ciencias Agrarias,1973-03-14,2015-11-23,51,2015,47-mayor,2010-2016,Diplomado en Educación Superior V.7,LORGIO SOLIZ BALDERAS,<humano> Dame una recomendacion de este curso ...,<titulo> Licenciado en Educación Ambiental <ar...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37652,37652,"MAYOR, REAL Y PONTIFICIA DE SAN FRANCISCO XAVIER",LUIS,TERRAZAS,GUERRA,Licenciado en Administración Agropecuaria,Diplomado en Gestión de Procesos de Contrataci...,73117327,Ciencias Agrarias,1986-02-25,2009-09-24,38,2009,33-39,antiguo,Diplomado en Gestión de Procesos de Contrataci...,LUIS TERRAZAS GUERRA,<humano> Dame una recomendacion de este curso ...,<titulo> Licenciado en Administración Agropecu...
37817,37817,"MAYOR, REAL Y PONTIFICIA DE SAN FRANCISCO XAVIER",BEIMAK,VARGAS,NEGRETE,Ingeniero Agrónomo,Diplomado en Gestión de Procesos de Contrataci...,77139743,Ciencias Agrarias,1971-01-01,2019-03-29,53,2019,47-mayor,2017-actual,Diplomado en Gestión de Procesos de Contrataci...,BEIMAK VARGAS NEGRETE,<humano> Dame una recomendacion de este curso ...,<titulo> Ingeniero Agrónomo <area> Ciencias Ag...
37818,37818,"MAYOR, REAL Y PONTIFICIA DE SAN FRANCISCO XAVIER",BEIMAK,VARGAS,NEGRETE,Ingeniero Agrónomo,"Diplomado en educación militar superior, Versi...",77139743,Ciencias Agrarias,1971-01-01,2019-03-29,53,2019,47-mayor,2017-actual,"Diplomado en educación militar superior, Versi...",BEIMAK VARGAS NEGRETE,<humano> Dame una recomendacion de este curso ...,<titulo> Ingeniero Agrónomo <area> Ciencias Ag...
37875,37875,AUTONOMA GABRIEL RENE MORENO,BRYAN ABATT,LIMON,BARRON,Licenciado en Medicina Veterinaria y Zootecnia,Especialidad en Cirugía y Traumatología Oral y...,78145413,Ciencias Agrarias,1995-12-15,2023-09-10,29,2023,menor-32,2017-actual,Especialidad en Cirugía y Traumatología Oral y...,BRYAN ABATT LIMON BARRON,<humano> Dame una recomendacion de este curso ...,<titulo> Licenciado en Medicina Veterinaria y ...


In [20]:
df.columns

Index(['index', 'Unnamed: 0', 'nombreuniversidad', 'nombres', 'paterno',
       'materno', 'nombretitulo', 'nombrecarrera', 'celular', 'nombrearea',
       'fechanacimiento', 'fechatitulacion', 'edad', 'egreso', 'edad_rango',
       'rango_ano_titulacion', 'nombrereemplazados', 'nombre_completo',
       'pregunta', 'respuesta', 'text'],
      dtype='object')

In [29]:
df['nombretitulo'].value_counts

<bound method IndexOpsMixin.value_counts of 0    Técnico Superior en Laboratorio Clinico
1                Contador Público Autorizado
2                                 Odontólogo
3                            Médico Cirujano
4            Ingeniero en Recursos Naturales
5                                 Odontólogo
6                                    Abogado
7                            Médico Cirujano
8                                 Bioquimico
9                Contador Público Autorizado
Name: nombretitulo, dtype: object>

nombretitulo
Abogado                                                        [Humanidades y Sociales]
Bioquimico                                                       [Ciencias de la Salud]
Contador Público Autorizado                  [Ciencias Económicas, Ciencias Económicas]
Ingeniero en Recursos Naturales                                     [Ciencias Agrarias]
Médico Cirujano                            [Ciencias de la Salud, Ciencias de la Salud]
Odontólogo                                 [Ciencias de la Salud, Ciencias de la Salud]
Técnico Superior en Laboratorio Clinico                          [Ciencias de la Salud]
Name: nombrearea, dtype: object

In [5]:
df['text'] = df['pregunta']+df['respuesta']

In [6]:
df = df[0:10]

In [7]:
df=df.reset_index()

In [8]:
nDf= df.loc[:,'text']

In [9]:
list(nDf)

['<humano> Dame una recomendacion de este curso <nombre>CURSO DE FORMACION CONTINUA EN: "INTENSIVO EN IDIOMA ORIGINARIO QUECHUA", VERSION II <asistente> <titulo> Técnico Superior en Laboratorio Clinico <area> Ciencias de la Salud <edad_rango> menor-32 <rango_ano_titulacion> 2017-actual <fin> ',
 '<humano> Dame una recomendacion de este curso <nombre>Especialización en Corrección Dental <asistente> <titulo> Contador Público Autorizado <area> Ciencias Económicas <edad_rango> 47-mayor <rango_ano_titulacion> antiguo <fin> ',
 '<humano> Dame una recomendacion de este curso <nombre>Especialización en Enderezamiento Dental <asistente> <titulo> Odontólogo <area> Ciencias de la Salud <edad_rango> 47-mayor <rango_ano_titulacion> antiguo <fin> ',
 '<humano> Dame una recomendacion de este curso <nombre>Diplomado en Educación Superior V.7 <asistente> <titulo> Médico Cirujano <area> Ciencias de la Salud <edad_rango> 47-mayor <rango_ano_titulacion> antiguo <fin> ',
 '<humano> Dame una recomendacion d

In [10]:
nDf[1]

'<humano> Dame una recomendacion de este curso <nombre>Especialización en Corrección Dental <asistente> <titulo> Contador Público Autorizado <area> Ciencias Económicas <edad_rango> 47-mayor <rango_ano_titulacion> antiguo <fin> '

In [11]:
dataset = Dataset.from_pandas(pd.DataFrame({'text': nDf}))

In [12]:
dataset

Dataset({
    features: ['text'],
    num_rows: 10
})

In [13]:
def tokenize_function(examples):
    outputs = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)
    outputs["labels"] = outputs["input_ids"].copy()
    return outputs


In [16]:
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=['text'])


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [17]:
tokenized_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 10
})

In [18]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # No queremos enmascarado de lenguaje para GPT-2
)
training_args = TrainingArguments(
    output_dir="./results",  # Directorio donde se guardarán los resultados
    overwrite_output_dir=True,
    num_train_epochs=1,  # Número de épocas de entrenamiento
    per_device_train_batch_size=2,  # Tamaño del lote de entrenamiento por dispositivo
    save_steps=1,  # Pasos después de los cuales guardar el modelo
    save_total_limit=2,
    weight_decay=0.01,
    remove_unused_columns=False
)

# Definir el objeto Trainer y entrenar el modelo
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator
)

# Entrenar el modelo


In [19]:
trainer.train()


Step,Training Loss


TrainOutput(global_step=5, training_loss=2.8685184478759767, metrics={'train_runtime': 122.328, 'train_samples_per_second': 0.082, 'train_steps_per_second': 0.041, 'total_flos': 2321751736320.0, 'train_loss': 2.8685184478759767, 'epoch': 1.0})